In [64]:
from qiskit_ibm_runtime import QiskitRuntimeService
import math
import numpy as np
import random
from qiskit import transpile
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import SamplerV2
from qiskit import QuantumCircuit
from math import gcd
import time

In [91]:
def modular_exponentiation(a, x, N, n):
    """
    Creates a quantum circuit instruction to perform modular exponentiation.

    Args:
        a (int): The base of the exponentiation.
        x (int): The power to which the base is raised.
        N (int): The modulus.
        n (int): Number of qubits required to represent the modulus.

    Returns:
        Instruction: A quantum circuit instruction for modular exponentiation.
    """
    qc = QuantumCircuit(n + 1)
    for i in range(n):
        qc.p(2 * np.pi * (a**(x % N)) / N, i)  # Phase rotation with modular exponentiation
    return qc.to_instruction()  # Return the quantum instruction to be reused


def inverse_qft(n):
    """
    Constructs an inverse Quantum Fourier Transform (QFT) circuit.

    Args:
        n (int): The number of qubits in the circuit.

    Returns:
        Instruction: A quantum circuit instruction for the inverse QFT.
    """
    qc = QuantumCircuit(n)
    for i in range(n // 2):
        qc.swap(i, n - i - 1)  # Swap qubits to reverse their order
    for i in range(n):
        for j in range(i):
            qc.cp(-np.pi / 2**(i - j), j, i)  # Controlled phase gate
        qc.h(i)  # Apply Hadamard gate
    return qc.to_instruction()  # Return the quantum instruction to be reused


def find_order_from_phase(phase_decimal):
    """
    Determines the order from the phase result of Quantum Phase Estimation (QPE).

    Args:
        phase_decimal (float): The phase result as a decimal value.

    Returns:
        int: The denominator of the phase fraction, which is the order.
    """
    fraction = phase_decimal.as_integer_ratio()  # Convert the phase into a fraction
    return fraction[1]  # The denominator gives the order


def qpe_setup(a, N, max_available_qubits=math.inf):
    """
    Generates a quantum circuit for the generalized phase estimation setup.

    Args:
        a (int): The base for modular exponentiation.
        N (int): The modulus.

    Returns:
        tuple: A tuple consisting of the quantum circuit (QuantumCircuit) and the
               number of phase estimation qubits (int).
               :param max_available_qubits:
    """
    n = math.ceil(math.log2(N))  # Number of qubits needed to represent N
    m = 2 * n  # Extra qubits for accuracy in the QPE

    if m > max_available_qubits:
        raise ValueError(f"Need {max_available_qubits} qubits for the quantum circuit but only have {m} available.")

    qc = QuantumCircuit(m + n, m)  # Create a circuit with m phase qubits and n target qubits

    qc.x(m)  # Set the first qubit of the target register to |1>

    # Initialize the phase qubits in superposition
    qc.h(range(m))  # Apply Hadamard gates to phase qubits

    # Apply controlled unitary operations
    for qubit in range(m):
        qc.append(modular_exponentiation(a, 2**qubit, N, n),
                  [qubit] + list(range(m, m + n)))  # Controlled evolution

    # Apply the inverse QFT to the phase register
    qc.append(inverse_qft(m), range(m))

    # Measure the phase qubits
    qc.measure(range(m), range(m))

    return qc, m


def qpe_ibm_quantum_backend(qc, m):
    """
    Implements Quantum Phase Estimation (QPE) to estimate the phase and determine the order using IBM's quantum computers.

    Args:
        a (int): The base for modular exponentiation.
        m: The number of qubits in the circuit.

    Returns:
        int: The order 'r' of the modular exponentiation.
    """

    print("Performing quantum phase estimation on IBM Quantum Computer...")
    print("Connecting to IBM Quantum Computer...")
    start_time = time.time()
    # Connect to IBM Quantum using Qiskit Runtime
    service = QiskitRuntimeService()
    backends = service.backends()
    backend = backends[0]
    execution_time = time.time() - start_time
    print(f"Connected to {backend}. Took {execution_time} seconds")

    # Access the number of qubits from the backend
    num_qubits = backend.num_qubits
    print(f"Number of qubits on quantum computer: {num_qubits}")

    # Run using the Sampler primitive
    transpiled_qc = transpile(qc, backend)
    sampler = SamplerV2(backend)
    start_time = time.time()
    print("Starting job on Quantum Computer...")
    job = sampler.run([transpiled_qc])
    result = job.result()
    execution_time = time.time() - start_time
    print(f"Execution time of job on Quantum Computer: {execution_time} seconds")

    # Extract phase estimation result using join_data and get_counts
    pub_result = result[0]
    counts = pub_result.join_data().get_counts()
    measured_value = max(counts, key=counts.get)
    phase_estimate = int(measured_value, 2) / (2**m)
    r = find_order_from_phase(phase_estimate)

    return r


def qpe_ibm_local_simulator(qc, m):
    """
    Implements Quantum Phase Estimation (QPE) to estimate the phase and determine the order using a local simulator.

    Args:
        a (int): The base for modular exponentiation.
        m: The number of qubits in the circuit.
        N (int): The modulus.
        backend (Backend): The backend to use.

    Returns:
        int: The order 'r' of the modular exponentiation.
    """

    simulator = AerSimulator()  # Use Qiskit's AerSimulator
    transpiled_qc = transpile(qc, simulator)  # Transpile the circuit for the simulator
    job = simulator.run(transpiled_qc)  # Run the simulation
    result = job.result()  # Get the result object

    # Extract phase estimation result
    counts = result.get_counts()
    measured_value = max(counts, key=counts.get)
    phase_estimate = int(measured_value, 2) / (2**m)
    r = find_order_from_phase(phase_estimate)

    return r

def shors_algorithm(N, run_locally):
    """
    Implements Shor's algorithm to find the non-trivial prime factors of a given number.

    Args:
        N (int): The number to factorize.

    Returns:
        tuple: A pair of non-trivial factors of 'n'.
    """
    if N % 2 == 0:  # Check if n is even
        return 2  # Return 2 as a factor

    # Pick a random number 'a' in the range [2, n-1]
    a = random.randint(2, N - 1)
    g = gcd(a, N)  # Compute the greatest common divisor (GCD) of a and n

    # If the gcd is not one then return it as a factor
    if g != 1:
        return g

    # Create the quantum circuit
    qc, m = qpe_setup(a, N)

    # Perform QPE to find the order 'r' of 'a' modulo 'n'
    if run_locally:
        r = qpe_ibm_local_simulator(qc, m)
    else:
        r = qpe_ibm_quantum_backend(qc, m)

    if r % 2 != 0 or pow(a, r // 2, N) == N - 1:  # Check if r is odd or invalid for factorization
        return shors_algorithm(N, run_locally)  # Retry with a different random 'a'

    # Compute the factors using the order 'r'
    factor1 = gcd(pow(a, r // 2) - 1, N)
    factor2 = gcd(pow(a, r // 2) + 1, N)

    if factor1 == 1 or factor2 == 1:  # If factors are trivial, retry
        return shors_algorithm(N, run_locally)

    return factor1, factor2  # Return the non-trivial factors of n

run_locally = True
N = 15

if run_locally:
    print(f"Running Shor's algorithm locally to find the non-trivial prime factors of {N}.")
    result = shors_algorithm(15, True)
    print(f"The non-trivial prime factors of {N} are {result}.")
else:
    print(f"Running Shor's algorithm on quantum computer to find the non-trivial prime factors of {N}.")
    result = shors_algorithm(N, False)
    print(f"The non-trivial prime factors of {N} are {result}.")

Running Shor's algorithm locally to find the non-trivial prime factors of 15.
The non-trivial prime factors of 15 are 3.


In [62]:
run_locally = False
N = 15

if run_locally:
    print(f"Running Shor's algorithm locally to find the non-trivial prime factors of {N}.")
    result = shors_algorithm(15, True)
    print(f"The non-trivial prime factors of {N} are {result}.")
else:
    print(f"Running Shor's algorithm on quantum computer to find the non-trivial prime factors of {N}.")

    result = shors_algorithm(N, False)
    print(f"The non-trivial prime factors of {N} are {result}.")


Running Shor's algorithm on quantum computer to find the non-trivial prime factors of 15.
Number of qubits on quantum computer: 127
Performing quantum phase estimation on IBM Quantum Computer...
Execution time of job on Quantum Computer: 0.9928159713745117 seconds


KeyboardInterrupt: 

In [92]:
from itertools import product
import sympy

def get_factors_from_prime_factors(prime_factors):
    """
    Get all factors of a number using its prime factorization.

    Args:
    - prime_factors (dict): A dictionary where keys are prime numbers and values are their exponents.
      Example: For 16 (2^4), pass {2: 4}.

    Returns:
    - List[int]: A sorted list of all factors of the number.
    """
    # Generate ranges for each prime factor based on its exponent
    factor_ranges = [[p ** e for e in range(exp + 1)] for p, exp in prime_factors.items()]

    # Use the Cartesian product to generate all combinations of factors
    all_factors = set(map(lambda x: eval('*'.join(map(str, x))), product(*factor_ranges)))

    # Return sorted list of factors
    return sorted(all_factors)

def get_all_factors(n):
    sympy_factors = sympy.factorint(n)
    return get_factors_from_prime_factors(sympy_factors)

[1, 2, 4, 5, 10, 20, 25, 50, 100]


In [121]:
for i in range(2, 161):
    actual_factors = get_all_factors(i)
    if len(actual_factors) == 2:
        continue
    shor_factors = shors_algorithm(i)
    if shor_factors is None:
        print("No factors found for", i)
    elif isinstance(shor_factors, tuple):
        print(f"Tuple found for {i}: {shor_factors}")
        for f in shor_factors:
            if f not in actual_factors:
                print(f"Factor {f} not found in actual factors {actual_factors} for {i}")
    else:
        if shor_factors not in actual_factors:
            print(f"Factor {shor_factors} not found in actual factors {actual_factors} for {i}")

In [115]:
def f():
    return 2, 3

result = f()
type_of_result = type(result)
print(type_of_result)
print(isinstance(result, tuple))

for val in f():
    print(val)

<class 'tuple'>
True
2
3


In [19]:
import Secrets
from qiskit_ibm_runtime import QiskitRuntimeService

# Save account to disk and save it as the default.
api_key = Secrets.IBM_QISKIT_API_KEY
instance="Main" #instance – The service instance to use. For ibm_cloud runtime, this is the Cloud Resource Name (CRN) or the service name. For ibm_quantum runtime, this is the hub/group/project in that format. https://quantum.cloud.ibm.com/
QiskitRuntimeService.save_account(channel="ibm_cloud", token=api_key, instance=instance, set_as_default=True, overwrite=True)

# Load the saved credentials
service = QiskitRuntimeService()

In [20]:
from qiskit import QuantumCircuit
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler

# Create empty circuit
example_circuit = QuantumCircuit(2)
example_circuit.measure_all()

backend = service.least_busy(operational=True, simulator=False)

sampler = Sampler(backend)
job = sampler.run([example_circuit])
print(f"job id: {job.job_id()}")

job id: d03l8sd1r6vs73dr1tsg
